## Diffusion from displacement of particles
In this part of the lab we are going to calculate the self-diffusion of water from data of the positions over time. In the program, a textfile called _msd.txt_ is generated after a simulation has been run, containing the mean squared displacement of water as a function of time. The mean squared displacement of water is calculated by summing the squares of the oxygen displacements from their respective reference positions determined by their start positions, $x_{0,n}$, at predefined points in time. Then, for each sampled point in time, the sum is divided by the total number of water molecules,
$$
\begin{align}
\langle(x(t)-x(t=0))^2\rangle = \frac{1}{N}\sum\limits_{n=1}^N (x_n(t)-x_n(t=0))^2
\label{msd}
\tag{1}
\end{align}
$$
where $N$ is the total number of particles. 

If particles interact, they will establish a random walk behaovior after enough "collisions" have ocuured, causing the mean squared displacement to increase lineary with time. However, in the start of a simulation, it is common that one observes a non-linear dependence of the mean squared displacement on time. This is known as the _ballistic_ regime, taking place because not enough collisions have occured to establish the random walk behavior. Mathematically, this can be shown by considering two different cases,


$$
\begin{equation}
x_1(t+\Delta t) = x(t) \pm \delta
\label{diff1}
\tag{2}
\end{equation}
$$   

$$
\begin{equation}
x_2(t+\Delta t) = x(t) + \delta
\label{diff2}
\tag{3}
\end{equation}
$$   
where equation \ref{diff1} describes random walk motion, with equal probabilities to move particles in the positive and negative direction, whereas equation \ref{diff2} describes ballistic motion, only displacing particles in the positive direction. The displacement parameter, $\delta$, is assumed to keep constant over time. 

1. Show mathematically that random walk diffusion increases linearly with time, whereas the ballistic one increases linearly with $t^2$, using equations \ref{diff1} and \ref{diff2}.

_Hint_: First consider one particle and sum over $n$ time steps. Then, average this sum by summing over $N$ particles. The first random walk mean squared displacement will show to increase linearly with time, whereas the ballistic one will increase linearly with $t^2$. The following relation might be useful: 

$$
\begin{equation}
\left(\sum_{i=1}^n h_i\right)^2 = \sum_{i=1}^n\sum_{j=1}^n h_ih_j
\end{equation}
$$

2. Run simulations with 216 water molecules for a series of temperatures (4-5 different, you choose which) and determine the diffusion coefficient of water for each temperature using the generated textfile _msd.txt_. Plot the mean squared displacements for all temperatures in the same plot, calculate the corresponding diffusion coefficients and report the values in a table. 

_Hints_: 

i) The diffusion coeficient can be obtained using the relation,
$$
\begin{equation}
D = \frac{\langle(x(t)-x_0)^2\rangle}{2 Nt}
\tag{4}
\end{equation}
$$

ii) The curve for mean squared displacement as a function of time is often not linear. To obtain a reasonable value of the diffusion coefficient, one most commonly makes a linear regression of the part of the curve that shows nearly linear behavior. This is done below using the package stats of scipy:

```python
slope, intercept, r_value, p_value, std_err = stats.linregress(time_array,msd_array)
```

In [ ]:
# Loading packages
import matplotlib.pyplot as plt
import numpy as np
import shutil
import os
from scipy import stats
from tempfile import mkstemp
from os import fdopen, remove

In [ ]:
# Naming directories to be used
os.chdir('../md')
datadir = '../MD_lab/data/'

In [ ]:
# Defining function to replace the value of input parameters in the input file 'config.mo' 
def replace(file_path, pattern, subst):
    with open(file_path,"r+") as f:
        new_f = f.readlines()
        f.seek(0)
        for line in new_f:
            if pattern not in line:
                f.write(line)
            else:
                f.write(subst+"\n")
        f.truncate()

In [ ]:
Ts = [] # ENTER CODE HERE: Enter the temperatures you want to simulate, separated by a comma
iterations = 10000 # multiply this with the timestep (0.001 ps) and you get the simulation time in ps. Use iterations >= 5000

replace('config.mo', 'iter', 'iter '+str(iterations))

for T in Ts:
    print(T)
    replace('config.mo', 'temperature', 'temperature '+str(T))
    !./build/main WAT_216.equil.gro params_prep.mo config.mo
    os.rename('msd.txt', 'msd_T_'+str(T)+'.txt')
    shutil.move("msd_T_"+str(T)+".txt", datadir+"msd_T_"+str(T)+".txt")
    

In [ ]:
for T in Ts: # Looping over temperatures
    time = np.loadtxt(datadir+"msd_T_"+str(T)+".txt", usecols=0) # Loading time array for temperature T
    msd = np.loadtxt(datadir+"msd_T_"+str(T)+".txt", usecols=1) # Loading mean squared displacement for each time, averaged over all oxygens, for temperature T

    ### Plot the msd vs time
    plt.plot() # ENTER CODE HERE: enter arguments to plot and add labels with label="arbitraryLabelName"
plt.xlabel('$time$ / ps', fontsize=13) # Assigning x-axis label
plt.ylabel(r'$\langle(x(t)-x_0)^2\rangle$ / (nm$^2$ ps$^{-1})$', fontsize=13) # Assigning y-axis label
plt.legend() # Plotting legend
plt.savefig(datadir+"msd_vs_time.pdf") # Saving figure as pdf in subdirectory "data"

In [ ]:
i = 0 # Initializing counter
D = np.zeros([len(Ts)]) # Initializing Diffusion vector
N =  # ENTER CODE HERE: Number of dimensions (to be used in the Einstein relation)

for T in Ts: # Looping over temperatures
    time = np.loadtxt(datadir+"msd_T_"+str(T)+".txt", usecols=0) # Loading time array for temperature T
    msd = np.loadtxt(datadir+"msd_T_"+str(T)+".txt", usecols=1) # Loading mean squared displacement for each time, averaged over all oxygens, for temperature T
    slope, intercept, r_value, p_value, std_err = stats.linregress(time,msd)
    D[i] = # ENTER CODE HERE: enter the expression for diffusion according to eq. 4, using the slope calculated 
    print("D(T="+str(T)+" K) = ", D[i])
    i = i+1 # Increasing counter by one



3. Plot the diffusion coefficient as a function of temperature in units cm$^2$/s and K, respectively. Discuss what you observe. Why does diffusion increase with temperature? Use keywords like entropy/thermal energy, enthalpy and local minima.  

In [ ]:
plt.plot(Ts, D) # Enter arguments (temperature and diffusion)
plt.xlabel('$T$ / K')
plt.ylabel('$D$ / (cm$^2$ s$^{-1}$)')

## Numerical Errors in MD Simulations:
A computer can normally only handle 15 significant numbers. This results in a small error each time a 
calculation is performed. To get more insight into this phenomenon and it's implications we take advantage 
of the deterministic nature of MD. 

In this part you will run a short simulation, approx 5ps, of 12 ions initially placed in the xy-plane, using a structure file called "2d.gro". You should then use the last frame of this simulation to run another simulation but this time backwards, with the same number of steps as the first. Since MD is deterministic you should end up at exactly the same configuration as the one you started with. It is important that you choose the same number of steps in both simulations to be able to get a clear picture of the error and the error propagation.

Since the ions will only move in two dimensions you are able to plot the positions of one particle in a graph. 

Run forward simulation:

In [ ]:
iterations = 5000 # multiply this with the timestep (0.001 ps) and you get the simulation time in ps

replace('config.mo', 'iter',        'iter '        + str(iterations))
replace('config.mo', 'temperature', 'temperature ' + str(300))
replace('config.mo', 'timestep',    'timestep '    + str(0.001))

!./build/main WAT_216.equil.gro params_prep.mo config.mo

Run backward simulation:

In [ ]:
replace('config.mo', 'timestep',    'timestep '    + str(-0.001))

!./build/main WAT_216.equil.gro params_prep.mo config.mo

1. Compare the structure of the first frame in the first simulation and the last frame in the second simulation. 

2. Plot the positions of both simulations and compare.

3. Calculate Lyapunov exponent? Maybe too much....

4. Discuss